In [13]:
import numpy as np
import os
import cv2

In [16]:
path = "D:\\PycharmProjects\\SPI_QML\\untrain_QuGe\\deep-generative-prior\\data_zhai\\128\\128\\randomP"
images_patterns = os.listdir(path)
images_patterns.sort()
randomP = []
for i, image in enumerate(images_patterns):
    image_matrix = cv2.imread(path+"\\"+image, 0)
    image_matrix = cv2.resize(image_matrix, (128, 128))
    randomP.append(image_matrix)

randomP = np.stack(randomP, axis=0)

print(randomP.shape)



(4096, 128, 128)


In [17]:
print(randomP[0, ])

[[ 55  35  20 ... 106  93  75]
 [ 51  33  19 ... 100  88  70]
 [ 46  30  17 ...  91  80  64]
 ...
 [ 45  24  10 ... 104  89  68]
 [ 52  31  15 ... 109  94  74]
 [ 55  35  18 ... 109  95  76]]


In [18]:
np.save(path+"\\randomP.npy", randomP)

In [19]:
import torch
def test_code(orignal_image, patterns, data_array):
    orignal_image = torch.Tensor(orignal_image)
    # patterns = torch.Tensor(patterns, dtype=torch.float32)  # with shape (1024*3, 64, 64)
    # I_orginal = torch.Tensor([torch.sum( orignal_image*patt ) for patt in patterns])
    I_orginal = torch.sum(torch.sum(torch.multiply(patterns, orignal_image), dim=-1), dim=-1)
    # I_orginal = torch.unsqueeze(I_orginal, dim=1)
    print(I_orginal, flush=True)
    print(torch.squeeze(data_array))
    test_result = I_orginal - torch.squeeze(data_array) 
    return test_result

In [29]:
wing_128 = cv2.imread("D:\\PycharmProjects\\SPI_QML\\untrain_QuGe\\deep-generative-prior\\data_zhai\\shot\\wing_128.png",0)
wing_128 = np.resize(wing_128/255.0, (128, 128))
# wing_128 = wing_128/255.0

patterns = np.load(path + "\\randomp.npy" )[:1024, :, :]/255.0

data_array = np.load("D:\\PycharmProjects\\SPI_QML\\untrain_QuGe\\deep-generative-prior\\data_zhai\\128\\128\\buckets\\wing.npy")
data_array = data_array[:1024,]

test_result = test_code(wing_128, torch.Tensor(patterns), torch.Tensor(data_array))

print(test_result)
A = torch.mean(torch.square(test_result))

A/(128*128)

tensor([557.6591, 491.0468, 553.6347,  ..., 594.7700, 627.2446, 656.6575])
tensor([590.9124, 524.6450, 619.0330,  ..., 615.5419, 707.9060, 681.6646])
tensor([-33.2533, -33.5983, -65.3983,  ..., -20.7719, -80.6614, -25.0071])


tensor(0.1360)

In [21]:
def DGI(signals, speckles):
    """
    """
    print(signals.shape)
    s_avg = np.mean(signals)
    r_avg = np.mean(speckles, 0)
    r_avg_avg = np.mean(speckles)
    N = len(signals) if len(signals) == len(speckles) else None
    object = np.zeros_like(speckles[0])

    for i in range(N):
        object += (signals[i] - (s_avg / r_avg_avg) * np.mean(speckles[i])) * (speckles[i] - r_avg)
    return object / N

In [22]:
image = DGI(np.squeeze(data_array), patterns)
# data_array (4096, 1)
# pattern (4096, 128, 128)
print(image)
image = (image - np.min(image)) / (np.max(image) - np.min(image))
print(image)
cv2.imwrite("filename.png", image*255)

(4096,)
[[-0.22820636 -0.25174393 -0.26616795 ... -0.13924943 -0.16790941
  -0.19847432]
 [-0.23249046 -0.25262537 -0.26495795 ... -0.15488244 -0.18069539
  -0.20701116]
 [-0.23293474 -0.24872666 -0.25844976 ... -0.16950129 -0.19093216
  -0.21251   ]
 ...
 [-0.20078414 -0.22891372 -0.24830502 ... -0.10035811 -0.13152766
  -0.16688011]
 [-0.21010673 -0.23847304 -0.25646487 ... -0.1088385  -0.14080611
  -0.17664621]
 [-0.21995236 -0.24707377 -0.26317157 ... -0.12342391 -0.15373399
  -0.18799329]]
[[0.06405082 0.027006   0.00430461 ... 0.20405656 0.15894976 0.11084489]
 [0.05730825 0.02561873 0.00620897 ... 0.17945239 0.13882643 0.0974091 ]
 [0.05660902 0.03175475 0.01645196 ... 0.15644438 0.12271519 0.08875469]
 ...
 [0.10720955 0.06293755 0.03241836 ... 0.26526604 0.21620954 0.16056975]
 [0.09253711 0.04789252 0.01957589 ... 0.25191909 0.20160656 0.14519928]
 [0.07704147 0.03435618 0.00902049 ... 0.22896371 0.18125989 0.12734056]]


True